In [32]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import os
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split

# Loading
import numpy as np
from keras.utils import np_utils


# Load the training set
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files)
print("Loading " + str(n) + " images")
imgs = np.asarray([mpimg.imread(image_dir + files[i]) for i in range(n)])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = np.asarray([mpimg.imread(gt_dir + files[i]) for i in range(n)])


x_train, x_test, y_train, y_test = train_test_split(imgs, gt_imgs, test_size=0.2)
print(x_train.shape, y_train.shape)

# input image dimensions
img_dim = 400
div = 1
if img_dim % div != 0:
    print("Invalid divider for the image dimensions!")
img_rows, img_cols = img_dim//div, img_dim//div

x_train = x_train.reshape(x_train.shape[0]*div*div, img_rows, img_cols, 3)
x_test = x_test.reshape(x_test.shape[0]*div*div, img_rows, img_cols, 3)
    
y_train = y_train.reshape(y_train.shape[0]*div*div, img_rows, img_cols)
y_test = y_test.reshape(y_test.shape[0]*div*div, img_rows, img_cols)

input_shape = (img_rows, img_cols, 3)
    
print(x_train.shape, y_train.shape)

batch_size = 128
num_classes = 2
epochs = 200

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

window_size = 72
patch_size = 16
input_shape = (window_size, window_size, 3)
padding = (window_size - patch_size) // 2

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
print(y_train.shape)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(64, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

def generate_minibatch(X, Y):
    """
    Procedure for real-time minibatch creation and image augmentation.
    This runs in a parallel thread while the model is being trained.
    """
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, window_size, window_size, 3))
        Y_batch = np.empty((batch_size, 2))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X.shape[0])
            shape = X[idx].shape

            # Sample a random window from the image
            center = np.random.randint(window_size//2, shape[0] - window_size//2, 2)
            sub_image = X[idx][center[0]-window_size//2:center[0]+window_size//2,
                               center[1]-window_size//2:center[1]+window_size//2]
            gt_sub_image = Y[idx][center[0]-patch_size//2:center[0]+patch_size//2,
                                  center[1]-patch_size//2:center[1]+patch_size//2]

            # The label does not depend on the image rotation/flip (provided that the rotation is in steps of 90°)
            threshold = 0.25
            label = (np.array([np.mean(gt_sub_image)]) > threshold) * 1

            label = np_utils.to_categorical(label, num_classes)
            X_batch[i] = sub_image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

Loading 100 images
Loading 100 images
(80, 400, 400, 3) (80, 400, 400)
(80, 400, 400, 3) (80, 400, 400)
x_train shape: (80, 400, 400, 3)
80 train samples
20 test samples
(80, 400, 400, 2)


In [33]:
model.fit_generator(generate_minibatch(x_train, y_train),
                    steps_per_epoch=5,
                    epochs=epochs,
                    verbose=1)

Epoch 1/200
5/5 [==============================] - 4s 707ms/step - loss: 0.6279 - acc: 0.8422
Epoch 2/200
5/5 [==============================] - 3s 573ms/step - loss: 0.0764 - acc: 1.0000
Epoch 3/200
5/5 [==============================] - 3s 572ms/step - loss: 0.0060 - acc: 1.0000
Epoch 4/200
5/5 [==============================] - 3s 573ms/step - loss: 0.0027 - acc: 1.0000
Epoch 5/200
5/5 [==============================] - 3s 573ms/step - loss: 0.0019 - acc: 1.0000
Epoch 6/200
5/5 [==============================] - 3s 575ms/step - loss: 0.0012 - acc: 1.0000
Epoch 7/200
5/5 [==============================] - 3s 576ms/step - loss: 9.8577e-04 - acc: 1.0000
Epoch 8/200
5/5 [==============================] - 3s 576ms/step - loss: 8.9371e-04 - acc: 1.0000
Epoch 9/200
5/5 [==============================] - 3s 575ms/step - loss: 6.0954e-04 - acc: 1.0000
Epoch 10/200
5/5 [==============================] - 3s 577ms/step - loss: 5.0071e-04 - acc: 1.0000
Epoch 11/200
5/5 [=========================

5/5 [==============================] - 3s 585ms/step - loss: 6.4579e-07 - acc: 1.0000
Epoch 85/200
5/5 [==============================] - 3s 583ms/step - loss: 1.7307e-06 - acc: 1.0000
Epoch 86/200
5/5 [==============================] - 3s 584ms/step - loss: 1.5799e-06 - acc: 1.0000
Epoch 87/200
5/5 [==============================] - 3s 583ms/step - loss: 4.5449e-07 - acc: 1.0000
Epoch 88/200
5/5 [==============================] - 3s 585ms/step - loss: 5.1149e-07 - acc: 1.0000
Epoch 89/200
5/5 [==============================] - 3s 578ms/step - loss: 1.6679e-06 - acc: 1.0000
Epoch 90/200
5/5 [==============================] - 3s 578ms/step - loss: 8.2090e-07 - acc: 1.0000
Epoch 91/200
5/5 [==============================] - 3s 577ms/step - loss: 3.8734e-07 - acc: 1.0000
Epoch 92/200
5/5 [==============================] - 3s 577ms/step - loss: 1.9870e-06 - acc: 1.0000
Epoch 93/200
5/5 [==============================] - 3s 576ms/step - loss: 6.5455e-07 - acc: 1.0000
Epoch 94/200
5/5 [=====

5/5 [==============================] - 3s 582ms/step - loss: 2.7931e-07 - acc: 1.0000
Epoch 167/200
5/5 [==============================] - 3s 583ms/step - loss: 3.4431e-07 - acc: 1.0000
Epoch 168/200
5/5 [==============================] - 3s 586ms/step - loss: 4.9100e-07 - acc: 1.0000
Epoch 169/200
5/5 [==============================] - 3s 587ms/step - loss: 2.6282e-07 - acc: 1.0000
Epoch 170/200
5/5 [==============================] - 3s 588ms/step - loss: 1.5644e-06 - acc: 1.0000
Epoch 171/200
5/5 [==============================] - 3s 587ms/step - loss: 1.9548e-07 - acc: 1.0000
Epoch 172/200
5/5 [==============================] - 3s 588ms/step - loss: 3.0622e-07 - acc: 1.0000
Epoch 173/200
5/5 [==============================] - 3s 588ms/step - loss: 2.7428e-07 - acc: 1.0000
Epoch 174/200
5/5 [==============================] - 3s 588ms/step - loss: 2.0601e-07 - acc: 1.0000
Epoch 175/200
5/5 [==============================] - 3s 588ms/step - loss: 3.9229e-07 - acc: 1.0000
Epoch 176/200


In [34]:
import itertools

xy_test_window = list(itertools.islice(generate_minibatch(x_test, y_test), 10))
# TODO real testing and prediction of the patches
xxx = list(zip(*xy_test_window))
score = model.evaluate(xxx[0][0], xxx[1][0], verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.19209303762e-07
Test accuracy: 1.0


In [35]:
model.save_weights('wight.h5')

### Helper functions

In [38]:
import re

def classify(model, X):
    """
    Classify an unseen set of samples.
    This method must be called after "train".
    Returns a list of predictions.
    """
    # Subdivide the images into blocks
    #/!\ change the patch_size and padding elements to something real!
    img_patches = create_patches(X, patch_size, 16, padding)

    if K.image_dim_ordering() == 'th':
        img_patches = np.rollaxis(img_patches, 3, 1)

    # Run prediction
    Z = model.model.predict(img_patches)
    Z = (Z[:,0] < Z[:,1]) * 1

    # Regroup patches into images
    return group_patches(Z, X.shape[0])

def load_image(infilename):
    """ Load an image from disk. """
    return mpimg.imread(infilename)

def pad_image(data, padding):
    """
    Extend the canvas of an image. Mirror boundary conditions are applied.
    """
    if len(data.shape) < 3:
        # Greyscale image (ground truth)
        data = np.lib.pad(data, ((padding, padding), (padding, padding)), 'reflect')
    else:
        # RGB image
        data = np.lib.pad(data, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    return data
    
def img_crop_gt(im, w, h, stride):
    """ Crop an image into patches (this method is intended for ground truth images). """
    assert len(im.shape) == 2, 'Expected greyscale image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0,imgheight,stride):
        for j in range(0,imgwidth,stride):
            im_patch = im[j:j+w, i:i+h]
            list_patches.append(im_patch)
    return list_patches
    
def img_crop(im, w, h, stride, padding):
    """ Crop an image into patches, taking into account mirror boundary conditions. """
    assert len(im.shape) == 3, 'Expected RGB image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    im = np.lib.pad(im, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    for i in range(padding,imgheight+padding,stride):
        for j in range(padding,imgwidth+padding,stride):
            im_patch = im[j-padding:j+w+padding, i-padding:i+h+padding, :]
            list_patches.append(im_patch)
    return list_patches
    
def create_patches(X, patch_size, stride, padding):
    img_patches = np.asarray([img_crop(X[i], patch_size, patch_size, stride, padding) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3], img_patches.shape[4])
    return img_patches
    
def create_patches_gt(X, patch_size, stride):
    img_patches = np.asarray([img_crop_gt(X[i], patch_size, patch_size, stride) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3])
    return img_patches
    
def group_patches(patches, num_images):
    return patches.reshape(num_images, -1)

def extract_img_features(filename, stride):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size, stride, padding)
    X = np.asarray([img_patches[i] for i in range(len(img_patches))])
    return X


def mask_to_submission_strings(model, image_filename):
    """ Reads a single image and outputs the strings that should go into the submission file. """
    img_number = int(re.search(r"\d+", image_filename).group(0))
    Xi = load_image(image_filename)
    Xi = Xi.reshape(1, Xi.shape[0], Xi.shape[1], Xi.shape[2])
    Zi = classify(model, Xi)
    Zi = Zi.reshape(-1)
    patch_size = 16
    nb = 0
    print("Processing " + image_filename)
    for j in range(0, Xi.shape[2], patch_size):
        for i in range(0, Xi.shape[1], patch_size):
            label = int(Zi[nb])
            nb += 1
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def generate_submission(model, submission_filename, *image_filenames):
    """ Generate a .csv containing the classification of the test set. """
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(model, fn))

### Load the weigt file and generate submission file
This have to go on the run.py then

In [39]:
model.load_weights('wight.h5')

model.model.summary()

submission_filename = 'submission.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = 'test_set_images/test_'+str(i)+'/test_' + str(i) + '.png'
    image_filenames.append(image_filename)
    

generate_submission(model, submission_filename, *image_filenames)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5_input (InputLayer)  (None, 72, 72, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 68, 68, 64)        4864      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 66, 66, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 33, 33, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 33, 33, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 69696)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8921216   
__________